# Introduction

The problem is to provide information to someone to help with a decision of where to move within the US. The scope will be the top 100 cities in the US by population. If the person who is moving has an idea of a city they like, for example, a city like Minneapolis, and wants to know other cities like that one, a cluster analysis could help identify the other similar cities within the same cluster as Minneapolis. Conversely, if the person moving knows they do NOT like Minneapolis, they could avoid moving to cities within the same cluster as it.

# Data

I will use data from wikipedia to provide list of the top 100 cities, including location (latitude and longitude), population, land area, population density, and growth rate. This data will have to be manipulated and cleaned up, extra characters removed, and converted to numeric data. I will also plot the 100 city locations on a US map.

To this data, I will add additional data from Foursquare about the venues in each city. The venue categories will be the main point of comparison, specifically, the percent breakdown of each type of venue category. To do this, I'll have to convert the categories into dummies first.

With the completed data set, I will normalize all the columns of the dataframe (population, area, growth, density, venue categories) for preparation into cluster analysis. I will run a few different cluster models and evaluate performance of these. Finally, I will provide the cluster results of similar cities, as well as an updated color-coded map.

# Methodology

I started by scraping the Wikipedia page that lists US cities by population using pandas. 
https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population

The trouble with the dataframe was in removing all the garbage characters, parsing latitude and longitude, and changing the data types from strings to floats. There was one particularly insidious problem where a character looked like a negative sign, but was giving an error converting string to float. It took way too long to figure out that a '-' is NOT the same as a '−'. Side-by-side: −-. I also learned that the astype(float) function does not handle commas well!

<img src="https://github.com/richtimm/coursera/blob/master/Top%20100%20df.png?raw=true" width="500"/>

After mapping the cities' locations using folium, it was time to start downloading Foursquare venues. I was able to re-use a bunch of code from the Toronto neighborhoods/boroughs lab, except this time, I wanted to pull a much bigger sample size of venues and from a larger radius. I increased the radius of search to 32km (~20 miles), and increased the maximum venue limit to 2000. I would have gone larger, and I tried, but I found that the API call started choking on the increased size of the json. It would work most of the time, but even a 5% failure rate was enough to trip up when I needed 100 API calls. I could have designed a more robust method using try/except, but instead I just settled for a 2000 venue limit. It seemed like a decent sample size to characterize the types of establishments within a large radius of the city.

I briefly considered grouping Foursquare venues into higher-level categories, e.g. different restaurant types would all be grouped into the category "Food" as defined by the hierarchy of Foursquare categories, found here https://developer.foursquare.com/docs/build-with-foursquare/categories/. However, I didn't want cities to lose their unique "fingerprints" with regard to different ethnic food options, plus it would have been more work, so I abandoned this idea.

To prepare the dataframe (9907 rows x 7 columns) of venues for cluster analysis, the venue category strings needed to be converted to dummies, which converted strings to numeric data and expanded the size to 9907 rows x 371 columns. From there, the dummies could be summed (groupby city) to give the number of each venues in each of the 100 cities, which resulted in a dataframe 100 rows x 371 columns of venue data.

The final steps in preparing for cluster analysis included joining the city dataframe with the venue dataframe (ensuring that the rows/cities matched up), and then normalizing the data. I chose MinMaxScaler to normalize, as I did not want to minimize distortion due to any outliers.

For cluster analysis, I chose KMeans because of its simplicity, and then had to decide on what K value to use. I wanted to visualize how performance would change with increasing K values, so after a bit of Googling, I landed on using the sum of squared distances method, finding a nice tutorial here: https://blog.cambridgespark.com/how-to-determine-the-optimal-number-of-clusters-for-k-means-clustering-14f27070048f. This let me attempt to visualize the "elbow" of the curve where increasing K no longer makes a big improvement in the summed error. 

# Results

I was soon in for a big surprise! I started by exploring K values of 1 to 10, and didn't see any "elbow". Ok... Increase to 20, then 50, then finally K=100. Yes, even with 100 clusters for 100 cities, the summed error was STILL decreasing, relatively linearly. Clearly that method wasn't working out in this application. So I punted. 

<img src="https://github.com/richtimm/coursera/blob/master/not%20elbow%20plot.png?raw=true" alt="Not an Elbow to be seen" width="500"/>

I played with a few K values manually visualizing the clusters on a map. I was expecting to see some trends based on geography, and that actually worked, somewhat. Cities in the midwest seemed to group (dark blue), a lot of greens in the west, NY/NJ in its own, Minneapolis/St Paul in its own.

<img src="https://github.com/richtimm/coursera/blob/master/Top%20100%20cluster%20map.png?raw=true" alt="Cluster map" width="500"/>

Here is the breakdown:

Cities in cluster 0: Chesapeake[m], Corpus Christi, Virginia Beach[m]

Cities in cluster 1: Fremont, San Jose

Cities in cluster 2: Albuquerque, Baltimore[m], Boston, Charlotte, Chicago, Cincinnati, Cleveland, Denver[i], Detroit, Indianapolis[h], Irving, Kansas City, Louisville[l], Madison, Memphis, Milwaukee, Nashville[k], New Orleans[n], Philadelphia[e], Pittsburgh, Sacramento, St. Louis[m], Tampa, Tulsa, Washington[j], Wichita

Cities in cluster 3: Atlanta, Austin, Baton Rouge[s], Boise[r], Chula Vista, Dallas, Garland, Glendale, Henderson, Honolulu[b], Houston[3], Jacksonville[f], Los Angeles, North Las Vegas, Oakland, Phoenix, Plano, Portland, Richmond[m], Scottsdale, St. Petersburg, Tucson

Cities in cluster 4: Jersey City, New York[d], Newark

Cities in cluster 5: Anchorage[p], Chandler, El Paso, Fort Wayne, Gilbert[q], Mesa, Toledo

Cities in cluster 6: Anaheim, Arlington, Aurora, Bakersfield, Buffalo, Colorado Springs, Durham, Fort Worth, Fresno, Irvine, Laredo, Lincoln, Long Beach, Lubbock, Norfolk[m], Oklahoma City, Omaha, Orlando, Riverside, San Antonio, San Diego, San Francisco[g], Santa Ana, Seattle, Winston–Salem

Cities in cluster 7: Minneapolis, Saint Paul

Cities in cluster 8: Hialeah, Miami

Cities in cluster 9: Columbus, Greensboro, Las Vegas, Lexington[o], Raleigh, Reno, Spokane, Stockton

# Discussion

This was, admittedly, a very limited analysis of comparing factors that make cities either more or less like each other. There were many very important and relevant features I thought of that I excluded from analysis for the sake of time. And clearly, no one in their right mind is going to rely solely on a cluster analysis to determine cities that they should live.

A few of the factors that would make for better comparisons include the weather, cost of living, traffic, natural amenities, school quality (if kids are a factor), political leanings, and job opportunities. From a career/life/family perspective, these would all be at least equal to if not more important to life happiness than analyzing venues found on Foursquare.

In addition, no weighting was given to the various columns of the cluster analysis. Given the nearly 400 venue categories, the sheer number of these venue categories would diminish the effect of other factors listed above. In addition to weighting factors, another approach would be to group similar venues (e.g. restaurants or at least restaurants of a certain cuisine category) to each other prior to normalization. This would give a more balance set of inputs prior to clustering.

If this exercize was to be done for more than academic purposes, I would recommend including the other factors listed above to give a more comprehensive and holistic assessment of the problem.

# Conclusions

In conclusion, I would not put a whole lot of stock in this analysis. Definitely don't make any major life decisions based on this cluster analysis. Adding other factors for comparing cities listed in the Discussion section (weather, cost of living, etc), would be necessary to make the model worthwhile. It might also give more of an "elbow" shape when finding the optimal K value instead of the nearly-straight-line that was evident for KMeans.

That said, it was a valuable learning exercize for me. Scraping data from the web is relatively straightforward with Pandas, but then cleaning it up can be frustrating and time-consuming for a novice Python programmer like myself. 